In [1]:
#veri analizi için gerkli olan pandas kütüp.
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np

In [46]:
#mediapipe kütüphanesi içerisindeki çizim yapma olanağı fonksiyonu 
#ve holistic modeli(vücut duruşunu analiz edebilen model.)
mp_drawing = mp.solutions.drawing_utils
mp_holistic= mp.solutions.holistic

In [58]:
df = pd.read_csv('Sağ El Dataları.csv',encoding='unicode_escape')
df.head()
#elde ettiğimiz dataları data frame aktar ve örnekleme yap

,sinif,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z19,v19,x20,y20,z20,v20,x21,y21,z21,v21
0,acik,0.225088,1.044629,7.739540e-07,0,0.315400,0.962998,-0.035961,0,0.376819,...,-0.110307,0,0.170698,0.537782,-0.131636,0,0.213942,0.468773,-0.144289,0
1,acik,0.208643,1.012064,9.563261e-07,0,0.306593,0.942716,-0.035472,0,0.369034,...,-0.069082,0,0.173087,0.463767,-0.081141,0,0.206512,0.388251,-0.088690,0
2,acik,0.209160,1.010453,8.666879e-07,0,0.302437,0.928409,-0.035031,0,0.363148,...,-0.076422,0,0.168821,0.451982,-0.089087,0,0.199531,0.374267,-0.097066,0
3,acik,0.195755,1.003875,9.036769e-07,0,0.295285,0.918182,-0.029585,0,0.355819,...,-0.071676,0,0.131151,0.427862,-0.082521,0,0.154082,0.342488,-0.089210,0
4,acik,0.197779,1.010492,8.196512e-07,0,0.297196,0.919405,-0.029836,0,0.359779,...,-0.069778,0,0.128618,0.425370,-0.081350,0,0.150866,0.338325,-0.088611,0


In [59]:
X = df.drop('sinif', axis=1) # ilk sütün olan drop edilir ve özellikler X değişkenine aktarılır.
y = df['sinif'] # y değişkenine sınıflandırma değerleri atılır. (açık ve kapalı)
X,y

(            x1        y1            z1  v1        x2        y2        z2  v2  \
 0     0.225088  1.044629  7.739540e-07   0  0.315400  0.962998 -0.035961   0   
 1     0.208643  1.012064  9.563261e-07   0  0.306593  0.942716 -0.035472   0   
 2     0.209160  1.010453  8.666879e-07   0  0.302437  0.928409 -0.035031   0   
 3     0.195755  1.003875  9.036769e-07   0  0.295285  0.918182 -0.029585   0   
 4     0.197779  1.010492  8.196512e-07   0  0.297196  0.919405 -0.029836   0   
 ...        ...       ...           ...  ..       ...       ...       ...  ..   
 1941  0.261729  0.905157 -5.820000e-07   0  0.354265  0.797013  0.004655   0   
 1942  0.264001  0.901586 -5.960000e-07   0  0.356859  0.794152 -0.002705   0   
 1943  0.270799  0.897609 -6.360000e-07   0  0.361502  0.801193 -0.006744   0   
 1944  0.275491  0.894282 -5.530000e-07   0  0.370733  0.803010 -0.015539   0   
 1945  0.307774  0.908972 -3.030000e-07   0  0.398564  0.848731 -0.033794   0   
 
             x3        y3 

In [60]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier


In [61]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [62]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [63]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [64]:
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle 

In [65]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

lr 0.9982876712328768
rc 0.9982876712328768
rf 0.9982876712328768
gb 0.9965753424657534


In [66]:
with open('RightHand.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

In [68]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        results = holistic.process(image)
        
        
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
        )
        try:
            pose = results.right_hand_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            
            row = pose_row
            
            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            
            
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            cv2.putText(image, 'SINIF'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, 'TAHMIN'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except Exception as error:
            print(error)
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


acik [9.99979353e-01 2.06465286e-05]
acik [9.99979353e-01 2.06465286e-05]


C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


acik [9.99979353e-01 2.06465286e-05]
acik [9.99979353e-01 2.06465286e-05]


C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X 

...

acik [9.99979353e-01 2.06465286e-05]
acik [9.99979353e-01 2.06465286e-05]
acik

C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X 

 [9.99979353e-01 2.06465286e-05]
acik [9.99979353e-01 2.06465286e-05]
acik

C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


 [9.99979353e-01 2.06465286e-05]
